In [1]:
import numpy as np
import pandas as pd
import rasterio as rs
import geopandas as gpd
import os
from rasterio.mask import mask

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import pickle

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
from xgboost import XGBRegressor



ModuleNotFoundError: No module named 'rasterio'

In [ ]:
df = pd.read_csv('../data/training/training.csv')

In [3]:
def cal_error(dff, y_test, predictions):
    mae = mean_absolute_error(y_test, predictions)
    rmse = mean_squared_error(y_test, predictions,squared=False)
    var_pred = dff['pred'].var()
    corr = dff['ytest'].corr(dff['pred']) 
    bias = (dff['pred']-dff['ytest']).sum()/(dff['ytest'].sum()) *100
    
    return mae, rmse, var_pred, corr

def plot(dff, y_test):
    m, b = np. polyfit(dff['ytest'] , dff['pred'],1 )
    plt.subplots(1,1)
    plt.scatter(dff['ytest'],dff['pred'])
    # plt.title(i)
    plt.plot(y_test, m*y_test + b)
    plt.xlabel('Field')
    plt.ylabel('Predicted')

def gen_predictions(model, mytarget):
    X_train, X_test, y_train, y_test = train_test_split( input,mytarget, test_size=0.30, random_state=42)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    predictions = predictions.reshape(len(predictions),)
    dff = pd.concat([y_test.reset_index(drop=True),pd.DataFrame(predictions).rename(columns={0:'pred'})],axis=1)
    dff.columns = ['ytest','pred']

    return dff, y_test, predictions

In [10]:
# input
N_model = RandomForestRegressor(criterion='squared_error', max_depth = 8, max_features = 4, oob_score=True,)
pH_model = RandomForestRegressor(criterion='squared_error', max_depth = 8, max_features = 3, oob_score=True,)#KNeighborsRegressor(n_neighbors=4)
K_model = RandomForestRegressor(criterion='squared_error', max_depth = 9, max_features = 3, oob_score=True,)
P_model = RandomForestRegressor(criterion='squared_error', max_depth = 9, max_features = 4, oob_score=True,)#SVR(kernel='rbf', C=20,  )
OC_model = SVR(kernel='rbf', C=5,  )

scaler = StandardScaler()

models = [ P_model, N_model, OC_model, K_model, pH_model]
dropna_options = ['P', 'N', 'OC', 'K', 'pH']

In [ ]:
for model, dropna_opt in zip( models, dropna_options):
    
    df_ = df.dropna(subset= [dropna_opt])
    scaler = StandardScaler()
    input = df_[['B8', 'B4', 'B5', 'B11','B9', 'B1', 'SR_n2', 'SR_N', 'TBVI1', 'NDWI', 'NDVI_G']]
    target = df_[[dropna_opt]]

    X_train, X_test, y_train, y_test = train_test_split( input,target, test_size=0.30)
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    predictions = predictions.reshape(len(predictions),)
    dff = pd.concat([y_test.reset_index(drop=True),pd.DataFrame(predictions).rename(columns={0:'pred'})],axis=1)
    dff.columns = ['ytest','pred']

    # dff, y_test, predictions= gen_predictions(model, target)
    mae, rmse, var_pred, corr = cal_error(dff, y_test, predictions)
    
    print("Target:", target.columns.values[0], "=> mae", mae, " rmse", rmse, "var_pred", var_pred, "corr", corr)
    # plot(dff, y_test)
    scores = cross_val_score(model, input,target, cv=3, scoring='neg_root_mean_squared_error')
    print(scores)
    with open(f"../data/models/{dropna_opt}.pkl", 'wb') as src:
        pickle.dump(model, src)
    with open(f"../data/models/{dropna_opt}_slr.pkl","wb") as src:
        pickle.dump(scaler, src)
        print(dropna_opt)



In [ ]:
predictions